In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.model_selection import cross_val_score


files = ['result_kujou_sud/' + path for path in os.listdir('result_kujou_sud')]
count_vect = CountVectorizer(input='filename')
X_train_counts = count_vect.fit_transform(files)


tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

features = count_vect.get_feature_names()
print(features[0:100])

X_train_tfidf = X_train_tfidf.toarray()
print(X_train_tfidf.max())
all_X = X_train_tfidf
all_y = np.loadtxt('only_suuti_label.csv', delimiter = '\n', dtype = float)
#print(all_y)
#print(type(all_X))
#print(type(all_y))
print(all_X.shape)
#print(count_vect.vocabulary_)

['11０', '1０', '1００', '1０００', '21', '2０', '2００', '2０１７', '82０', 'お客様の声', 'お金', 'しろ', 'ずれ', 'みれ', 'アウディ', 'アクセラ', 'アップ', 'アドバイス', 'アルミホイール', 'アンケート', 'アール', 'イング', 'ウェブ', 'エスカレーション', 'カスタマー', 'キュー', 'クレジット', 'グループ', 'コスト', 'コメント', 'コメント欄', 'コース', 'コールセンター', 'サポート', 'サービス', 'システム', 'ショップ', 'ストップ', 'センター', 'ダイヤル', 'ダウン', 'テニス', 'テル', 'ディーラー', 'デメリット', 'データ', 'ドア', 'ナンバー', 'ノーカウント', 'バイク', 'バンパー', 'ファックス', 'フォロー', 'フリーコール', 'フリーダイヤル', 'プラン', 'プリウス', 'プレー', 'プロテクト', 'ペルテ', 'ページ', 'ホーム', 'ポリープ', 'ポロ', 'マーク', 'ミス', 'メリット', 'メール', 'レジャー', 'レター', 'ワン', '一律', '一括', '一方的', '一時停止', '一環', '一般', '一言', '上から目線', '上記', '上長', '下記', '不備', '不十分', '不可', '不在', '不安', '不快', '不満', '不親切', '不足', '中学生', '中断', '中止', '了承', '了解', '予定', '争点', '事務的', '事実関係']
0.7563927854802805
(79, 615)


In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import KernelPCA
from tqdm import tqdm

train_X, test_X, train_y, test_y = train_test_split(all_X, all_y,
                                                    test_size=0.2,
                                                    random_state=42)

pipe_svc = make_pipeline(StandardScaler(),KernelPCA(n_components = 7,kernel = "rbf", gamma = 15, random_state =1 ), SVC(random_state = 2))
param_range = [0.00001,0.0001,0.001,0.01,0.1,1.0,10.0,100.0,1000.0,10000.0]
params = [{"svc__C":param_range, "svc__kernel":["linear"]},{"svc__C":param_range, "svc__gamma":param_range, "svc__kernel":["rbf"]}]
gs = GridSearchCV(estimator = pipe_svc, param_grid = params, scoring = "accuracy", cv = 5, n_jobs = -1)
gs = gs.fit(all_X, all_y)
print("Gridsearch_CV_score: %.3f" % gs.best_score_)
print("Gridsearch_best_param: %s" % gs.best_params_)

clf  = gs.best_estimator_
clf.fit(train_X, train_y)
print("Test accuracy : %.3f" % clf.score(test_X, test_y))

# for i in gs.grid_scores_:
#     print(i)

/Users/iwanagahidetoshi/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Gridsearch_CV_score: 0.519
Gridsearch_best_param: {'svc__C': 1e-05, 'svc__kernel': 'linear'}
Test accuracy : 0.250


AttributeError: 'GridSearchCV' object has no attribute 'grid_scores_'

In [2]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import KernelPCA
from tqdm import tqdm

loo = LeaveOneOut()
entire_count = loo.get_n_splits(all_X) 
p = np.array([1.0])
n = np.array([0.0])
print(p[0])
best = []

for s in range(8,15):
    for l in [0.00001,0.0001,0.001,0.01,0.1,1.0,10.0,100.0,1000.0,10000.0,100000.0]:
        for c in [ 0.00001,0.0001,0.001,0.01,0.1,1.0,10.0,100.0,1000.0,10000.0,100000.0 ]:
            for g in [0.00001,0.0001,0.001,0.01,0.1,1.0,10.0,100.0,1000.0,10000.0,100000.0]:
                pipe  = make_pipeline(StandardScaler(), KernelPCA(n_components = s,kernel = "rbf",gamma =l, random_state = 1), SVC(C =c, gamma=g, kernel = "rbf",random_state = 2))
                fn=0
                fp=0
                tp=0
                tn=0
                correct_answer_count = 0 
                for train_index, test_index in loo.split(all_X):
                    #print(train_index, test_index)
                    x_train, x_test = all_X[train_index], all_X[test_index]
                    y_train, y_test = all_y[train_index], all_y[test_index]
                    pipe.fit(x_train, y_train)  # 学習させる
                    result = pipe.predict(x_test)  # テストデータからラベルを予測する
                    if result == y_test:  # ラベルと元々のラベルが一致していれば+1
                        correct_answer_count += 1
                    if result != y_test and result[0] == n[0]:
                        fn += 1
                    elif result != y_test and result[0] == p[0]:
                        fp +=1
                    elif result == y_test and result[0] == p[0]:
                        tp += 1
                    elif result == y_test and result[0] == n[0]:
                        tn += 1

                accuracy = (float(tp+tn)/float(tp+tn+fp+fn))
                true_positive_rate = (float(tp)/float(tp+fn))
                true_negative_rate = (float(tn)/float(tn + fp))
                false_positive_rate = (float(fp)/float(fp + tn))
                false_negative_rate = (float(fn)/float(fn + tp))
                if accuracy >0.6:
                    best.append([s,l,c])

                print("jigen:%s     gamma= %s    SVM_C = %s    " % (s,l,c))
                print("Accuracy:%s" % accuracy)
                print("TP_rate: %s" % true_positive_rate)
                print("TN_rate: %s" % true_negative_rate)
                print("FP_rate: %s" % false_positive_rate)
                print("FN_rate: %s" % false_negative_rate)
                rate = (float(correct_answer_count) / float(entire_count))  # 正解率を計算
                print("Accuracy : %s" % str(rate))  # 正解率を出力
                print("F1: %s" % (2*(true_positive_rate*(float(tp)/float(tp+fp)))/(true_positive_rate+(float(tp)/float(tp+fp)))))
                print("")


print(tp)
print("kpca_best_param: %s" %best)
scores = cross_val_score(pipe, all_X, all_y , cv= 25)
print("Cross-Validation scores: {}".format(scores))
print("Average score: {}".format(np.mean(scores)))


pred_y2 = pipe.predict(test_X)
print(test_y)
print(pred_y2)
print(f1_score(test_y, pred_y2, average='micro'))



#from sklearn.metrics import confusion_matrix
#print(confusion_matrix(test_y, pred_y2))
#if result != y_test and float(result) ==　0:
   #     fn += 1
#elif result != y_test and float(result) == 1:
   #     fp +=1
#elif result == y_test and float(result) ==1:
 #       tp += 1
#elif result == y_test and float(result) == 0:
           #     fn += 1

1.0
jigen:8     gamma= 1e-05    SVM_C = 1e-05    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 1e-05    SVM_C = 1e-05    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 1e-05    SVM_C = 1e-05    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 1e-05    SVM_C = 1e-05    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 1e-05    SVM_C = 1e-05    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 1e-05    SVM_C = 1e-05    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 

jigen:8     gamma= 1e-05    SVM_C = 0.1    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 1e-05    SVM_C = 0.1    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 1e-05    SVM_C = 0.1    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 1e-05    SVM_C = 0.1    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 1e-05    SVM_C = 0.1    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 1e-05    SVM_C = 0.1    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0

jigen:8     gamma= 1e-05    SVM_C = 1000.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 1e-05    SVM_C = 1000.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 1e-05    SVM_C = 1000.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 1e-05    SVM_C = 1000.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 1e-05    SVM_C = 1000.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 1e-05    SVM_C = 1000.0    
Accuracy:0.4050632911392405
TP_rate: 0.731707317

jigen:8     gamma= 0.0001    SVM_C = 1e-05    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 0.0001    SVM_C = 1e-05    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 0.0001    SVM_C = 1e-05    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 0.0001    SVM_C = 0.0001    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 0.0001    SVM_C = 0.0001    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 0.0001    SVM_C = 0.0001    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_r

jigen:8     gamma= 0.0001    SVM_C = 0.1    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 0.0001    SVM_C = 1.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 0.0001    SVM_C = 1.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 0.0001    SVM_C = 1.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 0.0001    SVM_C = 1.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:8     gamma= 0.0001    SVM_C = 1.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rat

...

jigen:9     gamma= 0.001    SVM_C = 10.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:9     gamma= 0.001    SVM_C = 10.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:9     gamma= 0.001    SVM_C = 10.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:9     gamma= 0.001    SVM_C = 10.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:9     gamma= 0.001    SVM_C = 10.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:9     gamma= 0.001    SVM_C = 10.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rat

jigen:9     gamma= 0.001    SVM_C = 10000.0    
Accuracy:0.43037974683544306
TP_rate: 0.4878048780487805
TN_rate: 0.3684210526315789
FP_rate: 0.631578947368421
FN_rate: 0.5121951219512195
Accuracy : 0.43037974683544306
F1: 0.4705882352941176

jigen:9     gamma= 0.001    SVM_C = 10000.0    
Accuracy:0.46835443037974683
TP_rate: 0.7317073170731707
TN_rate: 0.18421052631578946
FP_rate: 0.8157894736842105
FN_rate: 0.2682926829268293
Accuracy : 0.46835443037974683
F1: 0.5882352941176471

jigen:9     gamma= 0.001    SVM_C = 10000.0    
Accuracy:0.4936708860759494
TP_rate: 0.9512195121951219
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.04878048780487805
Accuracy : 0.4936708860759494
F1: 0.6610169491525423

jigen:9     gamma= 0.001    SVM_C = 10000.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:9     gamma= 0.001    SVM_C = 100000.0    
Accuracy:0.5443037974683544
TP_rate: 0.975609756097561
TN_rate: 0.07

jigen:9     gamma= 0.01    SVM_C = 0.001    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:9     gamma= 0.01    SVM_C = 0.001    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:9     gamma= 0.01    SVM_C = 0.001    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:9     gamma= 0.01    SVM_C = 0.001    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:9     gamma= 0.01    SVM_C = 0.001    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:9     gamma= 0.01    SVM_C = 0.01    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate

jigen:9     gamma= 0.01    SVM_C = 10.0    
Accuracy:0.43037974683544306
TP_rate: 0.12195121951219512
TN_rate: 0.7631578947368421
FP_rate: 0.23684210526315788
FN_rate: 0.8780487804878049
Accuracy : 0.43037974683544306
F1: 0.18181818181818182

jigen:9     gamma= 0.01    SVM_C = 10.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:9     gamma= 0.01    SVM_C = 10.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:9     gamma= 0.01    SVM_C = 10.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:9     gamma= 0.01    SVM_C = 100.0    
Accuracy:0.5189873417721519
TP_rate: 1.0
TN_rate: 0.0
FP_rate: 1.0
FN_rate: 0.0
Accuracy : 0.5189873417721519
F1: 0.6833333333333332

jigen:9     gamma= 0.01    SVM_C = 100.0    

ZeroDivisionError: float division by zero